# Ego net downloader

COSC2671 Social Media and Network Analytics

Lukas Krodinger, 3931415

In [2]:
from twitterClient import twitterClient
import tweepy
import math
import networkx as nx

In [3]:
client = twitterClient()

In [4]:
# Assigns a logarithmic value depending on the follower count
def calculate_importance(follower_count):
    if follower_count <= 0:
        follower_count = 1
    return math.ceil(math.log10(follower_count))

Parameters:

In [5]:
# Username of the ego
username = "CricketAus"

# Where to store the graph to
graphFile='ego.graphml'

# ego attributes
ego_id = client.get_user(username=username).data.id
ego_username = client.get_user(username=username).data.username
ego_name = client.get_user(username=username).data.name

# maximum number of results
maxResults = 100

# All user fields
all_user_fields = ["id","name", "username", "created_at", "description", "location", "entities", "pinned_tweet_id", "profile_image_url", "protected", "public_metrics", "url", "verified", "withheld"]

# Fields of interest
user_fields_of_interest = ["id","name","public_metrics"]

In [6]:
# Create graph
ego_graph = nx.DiGraph()
ego_graph.add_node(ego_username)

In [7]:
# Get followers of ego
followers = []
for tweet in tweepy.Paginator(client.get_users_followers, id=ego_id, max_results=maxResults, user_fields=all_user_fields).flatten(limit=maxResults):
    followers.append(tweet)

In [8]:
# Remove not interesting fields
for follower in followers:
    for key in list(follower.keys()):
        if key not in user_fields_of_interest:
            delattr(follower, key)

In [9]:
# Construct followers graph
for user in followers:
    follower_name = user.username
    followers_count = user.public_metrics.get('followers_count')
    following_count = user.public_metrics.get('following_count')
    tweet_count = user.public_metrics.get('tweet_count')
    ego_graph.add_node(follower_name, followers_count=followers_count, following_count=following_count, tweet_count=tweet_count, color_id=calculate_importance(followers_count))
    ego_graph.add_edge(ego_username, follower_name)

In [10]:
# Get users that the ego follows
following = []
for tweet in tweepy.Paginator(client.get_users_following, id=ego_id, max_results=maxResults, user_fields=all_user_fields).flatten(limit=maxResults):
    following.append(tweet)

In [11]:
# Construct following graph
for user in following:
    following_name = user.username
    followers_count = user.public_metrics.get('followers_count')
    following_count = user.public_metrics.get('following_count')
    tweet_count = user.public_metrics.get('tweet_count')
    ego_graph.add_node(following_name, followers_count=followers_count, following_count=following_count, tweet_count=tweet_count, color_id=calculate_importance(followers_count))
    ego_graph.add_edge(following_name, ego_username)

In [12]:
# Store the graph
with open(graphFile, 'wb') as fOut:
    nx.write_graphml(ego_graph, fOut)